In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

load_dotenv()
directory = os.getenv("output_dir")

In [8]:
model = AffinityPropagation()

# Clustering with AP.
def clustering(dir, output_dir, model, dataset):
    paths = os.listdir(dir)
    files = sorted([file for file in paths if file.endswith(".csv")])

    ap_res = {}
    for file in files:
        df = pd.read_csv(os.path.join(dir, file))

        if "Unnamed: 0" in df.columns:
            df.drop(df.columns[0], axis=1, inplace=True)
            df.set_index(df.columns[0], inplace=True)

        else:
            df.set_index(df.columns[0], inplace=True)

        df = df.fillna(df.median())

        # AP clustering.
        model.fit(df)
        labels = model.labels_
        labels = labels + 1
        ap_res[file.split("/")[-1].split(".")[0]] = list(labels)
        result = pd.DataFrame(ap_res).set_index(df.index)

    result.to_csv(os.path.join(output_dir, f'{dataset}.csv'), index=True)

    return result

# riverflow metrics clustering
datasets = ["func", "func_climate", "all_attributes"]

for dataset in datasets:
    dir = os.path.join(directory, dataset)
    outputdir = os.path.join(directory, "ap_results")
    clustering(dir, outputdir, model, dataset)